## PyTorch implementation of Deep Bilteral Learning for Real Time Image Enhancement

In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np
from PIL import Image
import os, sys, glob
# import matplotlib.pyplot as plt
#Torch Imports
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
size = (256, 256)
batch_size = 100
learning_rate = 0.01
num_epochs = 5

In [3]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, train, transform=None):
        #Init Function
        super(Dataset, self).__init__()
        self.root_dir = root_dir
        self.train = train
        self.transform = transform
        
        self.full_res = []
        self.low_res = []
        
        if (train):
            dir = self.root_dir + '/train/'
        else :
            dir = self.root_dir + '/test/'
        
        for img_path in glob.glob (dir + '*.jpg'):
            
            himage = Image.open (img_path)
            limage = himage.resize (size)
            
            self.full_res.append (himage)
            self.low_res.append (limage)

    def __len__(self):
        #Length function ?
        return len(self.full_res)

    def __getitem__(self, idx):
        #Accessor Function
        if self.transform is None:
            return (self.full_res[idx],self.low_res[idx])
        else:
            limg_transformed = self.transform(self.low_res[idx])
            himg_transformed =  self.transform(self.full_res[idx])
            return (himg_transformed, limg_transformed)
       

In [4]:
composed_transform = transforms.Compose([transforms.ToTensor()])
train_dataset = Dataset (root_dir = '../data', train = True, transform = composed_transform)
im, im2 = train_dataset.__getitem__(0)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [70]:
class LocalFeatureNet (nn.Module):
           
    def __init__(self):
        super (LocalFeatureNet, self).__init__()
        
        self.relu  = nn.ReLU (inplace = True)
        
        self.conv1 = nn.Conv2d (in_channels = 3,   out_channels = 8, kernel_size = 3, stride = 2, padding = 1)
        self.conv2 = nn.Conv2d (in_channels = 8,  out_channels = 16,  kernel_size = 3, stride = 2, padding = 1)
        self.conv3 = nn.Conv2d (in_channels = 16, out_channels = 32,  kernel_size= 3, stride = 2,padding = 1)
        self.conv4 = nn.Conv2d (in_channels = 32, out_channels = 64,  kernel_size = 3, stride = 2, padding = 1)
        
        
        self.localconv1 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 1, padding = 1)
        self.localconv2 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size= 3, stride = 1, padding = 1)
        
        self.globalconv1 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 2,padding = 1)
        self.globalconv2 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 2, padding = 1)
        
        self.globalfc1 = nn.Linear (1024, 256)
        self.globalfc2 = nn.Linear (256, 128)
        self.globalfc3 = nn.Linear (128, 64)
        self.linear = nn.Conv2d(in_channels = 64, out_channels = 96,  kernel_size = 1, stride = 1)
        
        # Pixel Wise Network
        self.pixelwise_bias = nn.Parameter (torch.rand(3, 1), requires_grad=True)
        self.pixelwise_weight = nn.Parameter (torch.eye(3), requires_grad = True)
        self.pixelwise_obias = nn.Parameter (torch.eye(1), requires_grad = True)
        
        self.relu_slopes = nn.Parameter(torch.rand(3,16), requires_grad = True)
        self.relu_shifts = nn.Parameter(torch.rand(16,3), requires_grad = True)
        
    def custom_relu(self,channel,value):
        print("value",value.size())
        size = (16,value.size()[0],value.size()[1],value.size()[2])
        size_alt = (1L,value.size()[0],value.size()[1],value.size()[2])
        print("size: ",size,"size_alt",size_alt)
        value = value.expand(size)
        
        print(self.relu_shifts[:,channel])
        a = self.relu_shifts[:,channel].clone()
        a = a.view(16,1,1,1)
#         print(a.data,a)
#         value = self.relu(value - a.repeat(size_alt))    
# Upper line is not working for some damn reason so I hard coded it for now.
#         print ("here ", a.repeat(1,1,1080,1920).size())
        value = self.relu(value - a.repeat(1,1,1080,1920))
        print("st ", value.size())
        print ("st2 ", self.relu_slopes.size())
        value = self.relu_slopes.matmul(value.view(1080,1920,16,1))
        print ("stt ", value.size())
        value = value.view(1,3,1080,1920)
        value = value[:,1,:,:]+value[:,2,:,:]+value[:,0,:,:]
        print ("stt2 ", value.size())
        return value
    def upsample(self, grid, bilat):
        
        
    
    def forward (self, h, l):
        
#         print (x.size())
        
        x = self.relu (self.conv1 (l))
#         print (x.size())
        x = self.relu (self.conv2 (x))
#         print (x.size())
        x = self.relu (self.conv3 (x))
#         print (x.size())
        x = self.relu (self.conv4 (x))
#         print (x.size())
#         print ("lel")
        y = self.localconv1 (x)
#         print (y.size())
        y = self.localconv2 (y)
        
        print (y.size())
        
        z = self.globalconv1 (x)
        z = self.globalconv2 (z)
        z = self.globalfc1 (z.view(1, -1))
        z = self.globalfc2 (z)
        z = self.globalfc3 (z)
        
        print (z.size())
        
        
        fused = self.relu(z.view(-1,64,1,1)+y)
        print (fused.size())
        
        lin = self.linear(fused)
        print (lin.size())
        
        bilat = lin.view(-1, 8, 3, 4, 16, 16)
        
        
#         bilat = self.bilateralGrid (fused)
        
        print (bilat.size())
        
#         return bilat

# pixel wise network
        for i in range(0,3):
            a = self.pixelwise_weight[i,:].view(1,3)
            b = h.unsqueeze(0).view(3,-1)
            print(a.size(),b.size())
            p = torch.mm(a,b)
            print(p.size())
            p = p.view(h.size()[0],h.size()[2],h.size()[3]) + self.pixelwise_bias[i]
            print(p.size())
#             p = torch.bmm(self.pixelwise_weight[i,:], h.unsqueeze(0).view(3,-1)).view(h.size()) + self.pixelwise_bias[i]
            print(i)
            p += self.custom_relu(i,p)
        p += self.pixelwise_obias
        print(p.size())
        grid = p
        
        bilat_new = self.upsample(grid, bilat)
        
        

In [71]:
model = LocalFeatureNet() 

# Add code for using CUDA here if it is available
use_gpu = False
if(torch.cuda.is_available()):
    use_gpu = True
    model.cuda()

# Loss function and optimizers
criterion = nn.CrossEntropyLoss()# Define cross-entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)# Use Adam optimizer, use learning_rate hyper parameter

In [72]:
def train():
    # Code for training the model
    # Make sure to output a matplotlib graph of training losses
    loss_arr = []
    for epoch in range(num_epochs):
        for i, (himage, limage) in enumerate(train_loader):  
            # Convert torch tensor to Variable
            himage = Variable(himage)
            limage = Variable(limage)
            if(use_gpu):
                himage=himage.cuda()
                limage=limage.cuda()
            # Forward + Backward + Optimize
            optimizer.zero_grad()  # zero the gradient buffer
            outputs = model(himage, limage)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             loss_arr.append(loss.data[0])
#             if (i+1) % batch_size == 0:       
#                 print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
#                        %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
    
#     plt.plot( np.array(range(1,len(loss_arr)+1)), np.array(loss_arr))
#     plt.show()


In [73]:
%time train()

torch.Size([1, 64, 16, 16])
torch.Size([1, 64])
torch.Size([1, 64, 16, 16])
torch.Size([1, 96, 16, 16])
torch.Size([1, 8, 3, 4, 16, 16])
torch.Size([1, 3]) torch.Size([3, 2073600])
torch.Size([1, 2073600])
torch.Size([1, 1080, 1920])
0
value torch.Size([1, 1080, 1920])
size:  (16, 1L, 1080L, 1920L) size_alt (1L, 1L, 1080L, 1920L)
Variable containing:
 0.9785
 0.8374
 0.5139
 0.8680
 0.4341
 0.5143
 0.1708
 0.1736
 0.9382
 0.6981
 0.1556
 0.5258
 0.5393
 0.7172
 0.4651
 0.8818
[torch.cuda.FloatTensor of size 16 (GPU 0)]

st  torch.Size([16, 1, 1080, 1920])
st2  torch.Size([3, 16])
stt  torch.Size([1080, 1920, 3, 1])
stt2  torch.Size([1, 1080, 1920])
torch.Size([1, 3]) torch.Size([3, 2073600])
torch.Size([1, 2073600])
torch.Size([1, 1080, 1920])
1
value torch.Size([1, 1080, 1920])
size:  (16, 1L, 1080L, 1920L) size_alt (1L, 1L, 1080L, 1920L)
Variable containing:
 0.1407
 0.5043
 0.0411
 0.2476
 0.7290
 0.6807
 0.0849
 0.2551
 0.6171
 0.2061
 0.1674
 0.5127
 0.2465
 0.7837
 0.1369
 0.5396

Variable containing:
 0.3193
 0.4542
 0.8621
 0.8624
 0.6558
 0.3360
 0.3351
 0.5211
 0.3874
 0.1961
 0.6558
 0.1022
 0.6392
 0.4775
 0.9145
 0.8435
[torch.cuda.FloatTensor of size 16 (GPU 0)]

st  torch.Size([16, 1, 1080, 1920])
st2  torch.Size([3, 16])
stt  torch.Size([1080, 1920, 3, 1])
stt2  torch.Size([1, 1080, 1920])
torch.Size([1, 1080, 1920])
CPU times: user 5 s, sys: 1.42 s, total: 6.43 s
Wall time: 6.34 s
